# Building Fast Queries on a CSV

The goal of this project is to employ various techniques and algorithms to speed up the processing of inventory data to answer business questions about a hypothetical online laptop store. The data set used was adapted from the [Laptop Prices dataset on Kaggle](https://www.kaggle.com/ionaskel/laptop-prices). Here is brief description of the rows:

- ID: A unique identifier for the laptop.
- Company: The name of the company who produces the laptop.
- Product: The name of the laptop.
- TypeName: The type of laptop.
- Inches: The size of the screen in inches.
- ScreenResolution: The resolution of the screen.
- CPU: The laptop CPU.
- RAM: The amount of RAM in the laptop.
- Memory: The size of the hard drive.
- GPU: The graphics card name.
- OpSys: The name of the operating system.
- Weight: The laptop weight.
- Price: The price of the laptop.

## Loading and Examining the Data

In [1]:
import csv

with open('laptops.csv') as file:
    opened_file = csv.reader(file)
    rows = list(opened_file)
    
header = rows[0]
rows = rows[1:]
print(rows[:5])

[['6571244', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', '1339'], ['7287764', 'Apple', 'Macbook Air', 'Ultrabook', '13.3', '1440x900', 'Intel Core i5 1.8GHz', '8GB', '128GB Flash Storage', 'Intel HD Graphics 6000', 'macOS', '1.34kg', '898'], ['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', '575'], ['9722156', 'Apple', 'MacBook Pro', 'Ultrabook', '15.4', 'IPS Panel Retina Display 2880x1800', 'Intel Core i7 2.7GHz', '16GB', '512GB SSD', 'AMD Radeon Pro 455', 'macOS', '1.83kg', '2537'], ['8550527', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 3.1GHz', '8GB', '256GB SSD', 'Intel Iris Plus Graphics 650', 'macOS', '1.37kg', '1803']]


## Creating an Inventory Class

The goal here is build an initial class called Inventory and make incremental improvements throughout this analysis. Here is the first build of the class:

In [2]:
class Inventory():
    
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            opened_file = csv.reader(file)
            rows = list(opened_file)
        self.header = rows[0]
        self.rows = rows[1:]
        
        for row in self.rows:
            price = int(row[-1])
            row[-1] = price

inventory = Inventory('laptops.csv')
print(inventory.header)
print(len(inventory.rows))

['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']
1303


## Adding a Search Feature by Laptop ID

In [3]:
class Inventory():
    
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            opened_file = csv.reader(file)
            rows = list(opened_file)
        self.header = rows[0]
        self.rows = rows[1:]
        
        for row in self.rows:
            price = int(row[-1])
            row[-1] = price
    
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None

inventory = Inventory('laptops.csv')
print(inventory.get_laptop_from_id('3362737'))
print(inventory.get_laptop_from_id('3362736'))

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]
None


## Preprocessing Data for Faster Searches: IDs to Dictionary
We can improve the performance by preprocessing the data into a dictionary that uses the ID as keys and the rows for the values:

In [4]:
class Inventory():
    
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            opened_file = csv.reader(file)
            rows = list(opened_file)
        self.header = rows[0]
        self.rows = rows[1:]
        self.id_to_row = {}
        
        for row in self.rows:
            price = int(row[-1])
            row[-1] = price
            # assign key/value pairs in dictionary
            self.id_to_row[row[0]] = row
            
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None          
            
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None

inventory = Inventory('laptops.csv')
print(inventory.get_laptop_from_id_fast('3362737'))
print(inventory.get_laptop_from_id_fast('3362736'))

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]
None


The above initialization to preprocess the data takes the `get_laptop_from_id()` method from a time complexity of *O(R)* to *O(1)*. It accomplishes this by using more memory to store the `self.id_to_row` dictionary and more time to initialize the object up front. 

Let's see about comparing the performance of both functions. To do this, we will utilize the `random` and `time` modules to compare the execution times of queries assigned to each method.

## Comparing Performance of Method Calls

In [5]:
import random, time

ids = [str(random.randint(1000000, 9999999)) for _ in range(10000)]
print(ids[:10])

['8329891', '4961154', '9795453', '6461540', '1315973', '5760984', '6174959', '7205331', '4267130', '8895828']


In [6]:
total_time_no_dict = 0
total_time_dict = 0

# collect total time for no dictionary method
for i in ids:
    start = time.time()
    inventory.get_laptop_from_id(i)
    end = time.time()
    duration = end - start
    total_time_no_dict += duration

# collect total time for dictionary method
for i in ids:
    start = time.time()
    inventory.get_laptop_from_id_fast(i)
    end = time.time()
    duration = end - start
    total_time_dict += duration
    
print(total_time_no_dict)
print(total_time_dict)

1.2555334568023682
0.0059032440185546875


As we can see, there is a significant performance boost to using a dictionary. Dividing 1.2813 by 0.0055, it is shown that the second method is 237 times faster for the same input size.

## Adding a Gift Card Amount Check Feature

Let's add a method call to determine whether a given single use only gift card can be spent fully (i.e. no credit left over) by purchasing up to two laptops.

In [7]:
class Inventory():
    
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            opened_file = csv.reader(file)
            rows = list(opened_file)
        self.header = rows[0]
        self.rows = rows[1:]
        self.id_to_row = {}
        
        for row in self.rows:
            price = int(row[-1])
            row[-1] = price
            # assign key/value pairs in dictionary
            self.id_to_row[row[0]] = row
            
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None          
            
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if row[-1] == dollars:
                return True
        for row1 in self.rows:
            for row2 in self.rows:
                if row1[-1] + row2[-1] == dollars:
                    return True
        return False
    
inventory = Inventory('laptops.csv')
print(inventory.check_promotion_dollars(1000))
print(inventory.check_promotion_dollars(442))

True
False


## Preprocessing Data for Faster Searches: Prices to Set

In [8]:
class Inventory():
    
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            opened_file = csv.reader(file)
            rows = list(opened_file)
        self.header = rows[0]
        self.rows = rows[1:]
        self.id_to_row = {}
        self.prices = set()
        
        for row in self.rows:
            price = int(row[-1])
            row[-1] = price
            # add prices to set
            self.prices.add(row[-1])
            # assign key/value pairs in dictionary
            self.id_to_row[row[0]] = row
            
            
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None          
            
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if row[-1] == dollars:
                return True
        for row1 in self.rows:
            for row2 in self.rows:
                if row1[-1] + row2[-1] == dollars:
                    return True
        return False
    
    def check_promotion_dollars_fast(self, dollars):
        if dollars in self.prices:
                return True
        for price in self.prices:
            if dollars - price in self.prices:
                return True
        return False
    
inventory = Inventory('laptops.csv')
print(inventory.check_promotion_dollars_fast(1000))
print(inventory.check_promotion_dollars_fast(442))

True
False


## Comparing Performance of Promotion Functions

In [9]:
prices = [random.randint(100, 5000) for _ in range(100)]
print(prices[:10])

[1150, 134, 2918, 2352, 4288, 3122, 4269, 709, 4996, 3310]


In [10]:
total_time_no_set = 0
total_time_set = 0

# collect total time for no dictionary method
for i in prices:
    start = time.time()
    inventory.check_promotion_dollars(i)
    end = time.time()
    duration = end - start
    total_time_no_set += duration

# collect total time for dictionary method
for i in prices:
    start = time.time()
    inventory.check_promotion_dollars_fast(i)
    end = time.time()
    duration = end - start
    total_time_set += duration
    
print(total_time_no_set)
print(total_time_set)

1.7713029384613037
0.0008859634399414062


Again there is a significant speed up in performance when using an initializing set--about 1999 times faster for the same input size.

## Adding a Function to Find Laptops Based on Budget

Let's add a new feature to set budgetary limits that can find laptops that fall within that range:

In [11]:
class Inventory():
    
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            opened_file = csv.reader(file)
            rows = list(opened_file)
        self.header = rows[0]
        self.rows = rows[1:]
        self.id_to_row = {}
        self.prices = set()
        
        for row in self.rows:
            price = int(row[-1])
            row[-1] = price
            # add prices to set
            self.prices.add(row[-1])
            # assign key/value pairs in dictionary
            self.id_to_row[row[0]] = row
            
        self.rows_by_price = sorted(self.rows, key=lambda row: row[-1])
            
            
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None          
            
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if row[-1] == dollars:
                return True
        for row1 in self.rows:
            for row2 in self.rows:
                if row1[-1] + row2[-1] == dollars:
                    return True
        return False
    
    def check_promotion_dollars_fast(self, dollars):
        if dollars in self.prices:
                return True
        for price in self.prices:
            if dollars - price in self.prices:
                return True
        return False
    
    def find_laptop_with_price(self, target_price):
        range_start = 0                                   
        range_end = len(self.rows_by_price) - 1                       
        while range_start < range_end:
            range_middle = (range_end + range_start) // 2  
            price = self.rows_by_price[range_middle][-1]
            if price == target_price:                            
                return range_middle                        
            elif value < target_value:                           
                range_start = range_middle + 1             
            else:                                          
                range_end = range_middle - 1 
        price = self.rows_by_price[range_start][-1]
        if price != target_price:                  
            return -1                                      
        return range_start
    
    def find_first_laptop_more_expensive(self, target_price):
        range_start = 0                                   
        range_end = len(self.rows_by_price) - 1                       
        while range_start < range_end:
            range_middle = (range_end + range_start) // 2  
            price = self.rows_by_price[range_middle][-1]
            if price > target_price:                            
                range_end = range_middle                                    
            else:                                          
                range_start = range_middle + 1 
        if self.rows_by_price[range_start][-1] <= target_price:                  
            return -1                                      
        return range_start    
    
inventory = Inventory('laptops.csv')
print(inventory.find_first_laptop_more_expensive(1000))
print(inventory.find_first_laptop_more_expensive(10000))

683
-1
